In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.regularizers import l2

def improved_tinyml_mobilenet():
    input_shape = (96, 96, 1)  # Grayscale Input
    num_classes = 1  # Binary classification (Dog vs Not-Dog)
    num_filters = 16  # Start with more filters

    inputs = Input(shape=input_shape)

    # **1st Layer: Convolution**
    x = Conv2D(num_filters, (3,3), strides=2, padding='same', 
               kernel_initializer='he_normal', kernel_regularizer=l2(1e-5), use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)  # ✅ Swish works better than ReLU

    # **2nd Layer: Depthwise Separable Convolution**
    x = DepthwiseConv2D((3,3), strides=1, padding='same', 
                         depthwise_initializer='he_normal', depthwise_regularizer=l2(1e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)

    num_filters *= 2  # Increase filters
    x = Conv2D(num_filters, (1,1), strides=1, padding='same', 
               kernel_initializer='he_normal', kernel_regularizer=l2(1e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)

    # **3rd Layer: Depthwise Separable Convolution**
    x = DepthwiseConv2D((3,3), strides=2, padding='same', 
                         depthwise_initializer='he_normal', depthwise_regularizer=l2(1e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)

    num_filters *= 2  # Increase filters
    x = Conv2D(num_filters, (1,1), strides=1, padding='same', 
               kernel_initializer='he_normal', kernel_regularizer=l2(1e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)

    # **4th Layer: Depthwise Separable Convolution**
    x = DepthwiseConv2D((3,3), strides=1, padding='same', 
                         depthwise_initializer='he_normal', depthwise_regularizer=l2(1e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)

    # **5th Layer: Depthwise Separable Convolution**
    x = DepthwiseConv2D((3,3), strides=2, padding='same', 
                         depthwise_initializer='he_normal', depthwise_regularizer=l2(1e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)

    num_filters *= 2  # Increase filters
    x = Conv2D(num_filters, (1,1), strides=1, padding='same', 
               kernel_initializer='he_normal', kernel_regularizer=l2(1e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('swish')(x)

    # **Global Average Pooling**
    x = GlobalAveragePooling2D()(x)

    # **Fully Connected Layers**
    x = Dense(256, activation='swish', kernel_regularizer=l2(1e-5))(x)  # ✅ Increased neurons
    x = Dropout(0.4)(x)  # ✅ Increased dropout for better regularization
    x = Dense(128, activation='swish', kernel_regularizer=l2(1e-5))(x)
    x = Dropout(0.3)(x)

    # **Output Layer**
    outputs = Dense(1, activation='sigmoid')(x)  # ✅ Binary classification (Dog vs Not-Dog)

    # **Define Model**
    model = Model(inputs=inputs, outputs=outputs)

    return model

# **Compile the Model**
model = improved_tinyml_mobilenet()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), 
              loss="binary_crossentropy", metrics=["accuracy"])

# **Display Model Summary**
model.summary()

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from sklearn.utils.class_weight import compute_class_weight

# Image size and batch size
IMG_SIZE = (96, 96)
BATCH_SIZE = 16  # 🔥 Reduced for better generalization
EPOCHS = 30  # Will stop early if needed

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Compute class weights based on dataset size
class_counts = np.array([11438 + 2818, 23947 + 6029])  # Total count for each class
classes = np.array([0, 1])  # Labels: Not-Dog (0), Dog (1)

class_weights = compute_class_weight("balanced", classes=classes, y=np.repeat(classes, class_counts))
class_weights = {i: w for i, w in enumerate(class_weights)}

# 🔥 Reduce imbalance effect slightly
class_weights[0] *= 0.8  # Reduce Not-Dog weight
class_weights[1] *= 1.2  # Increase Dog weight
print(f"Updated Class Weights: {class_weights}")

# 🔥 Learning Rate Scheduler (decays LR every 5 epochs)
def lr_scheduler(epoch, lr):
    if epoch > 5:
        return lr * 0.8  # Reduce LR by 20% every 5 epochs
    return lr

lr_callback = LearningRateScheduler(lr_scheduler)

# Define Model
model = improved_tinyml_mobilenet()  # Call your updated model function

# Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),  # 🔥 Reduced LR for smoother learning
    loss="binary_crossentropy", 
    metrics=["accuracy"]
)

# Train the model
history = model.fit(
    train_images, train_labels,
    validation_data=(val_images, val_labels),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weights,  # Apply class balancing
    callbacks=[early_stopping, lr_callback]  # 🔥 Include LR scheduler
)

# Save model
MODEL_SAVE_PATH = "New_Model_Update.h5"
model.save(MODEL_SAVE_PATH)
print(f"✅ Model training complete! Saved as {MODEL_SAVE_PATH}")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight  # Correct import


# Image size and batch size
IMG_SIZE = (96, 96)
BATCH_SIZE = 32
EPOCHS = 30  # Will stop early if needed

# Define early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

# Compute class weights based on dataset size
class_counts = np.array([11438 + 2818, 23947 + 6029])  # Total count for each class
classes = np.array([0, 1])  # Labels: Not-Dog (0), Dog (1)

class_weights = compute_class_weight("balanced", classes=classes, y=np.repeat(classes, class_counts))
class_weights = {i: w for i, w in enumerate(class_weights)}

# Print computed class weights
print(f"Computed Class Weights: {class_weights}")

# Define Model
model = improved_tinyml_mobilenet()  # Call your updated model function

# Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
    loss="binary_crossentropy", 
    metrics=["accuracy"]
)

# Train the model
history = model.fit(
    train_images, train_labels,
    validation_data=(val_images, val_labels),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weights,  # Apply class balancing
    callbacks=[early_stopping]
)

# Save model
MODEL_SAVE_PATH = "New_Model_Update.h5"
model.save(MODEL_SAVE_PATH)
print(f"✅ Model training complete! Saved as {MODEL_SAVE_PATH}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.regularizers import l2

def improved_tinyml_mobilenet():
    input_shape = (96, 96, 1)  # Grayscale Input
    num_classes = 1  # Binary classification (Dog vs Not-Dog)
    num_filters = 16  # 🔥 Increased from 8 to 16

    inputs = Input(shape=input_shape)

    # **1st Layer: Convolution**
    x = Conv2D(num_filters, (3,3), strides=2, padding='same', 
               kernel_initializer='he_normal', kernel_regularizer=l2(5e-5), use_bias=False)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  # ✅ Using ReLU

    # **2nd Layer: Depthwise Separable Convolution**
    x = DepthwiseConv2D((3,3), strides=1, padding='same', 
                         depthwise_initializer='he_normal', depthwise_regularizer=l2(5e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  # ✅ Using ReLU

    num_filters *= 2  # 🔥 Increase filters
    x = Conv2D(num_filters, (1,1), strides=1, padding='same', 
               kernel_initializer='he_normal', kernel_regularizer=l2(5e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  # ✅ Using ReLU

    # **3rd Layer: Depthwise Separable Convolution**
    x = DepthwiseConv2D((3,3), strides=2, padding='same', 
                         depthwise_initializer='he_normal', depthwise_regularizer=l2(5e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  # ✅ Using ReLU

    num_filters *= 2  # 🔥 Increase filters
    x = Conv2D(num_filters, (1,1), strides=1, padding='same', 
               kernel_initializer='he_normal', kernel_regularizer=l2(5e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  # ✅ Using ReLU

    # **4th Layer: Depthwise Separable Convolution**
    x = DepthwiseConv2D((3,3), strides=1, padding='same', 
                         depthwise_initializer='he_normal', depthwise_regularizer=l2(5e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  # ✅ Using ReLU

    # **5th Layer: Depthwise Separable Convolution**
    x = DepthwiseConv2D((3,3), strides=2, padding='same', 
                         depthwise_initializer='he_normal', depthwise_regularizer=l2(5e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  # ✅ Using ReLU

    num_filters *= 2  # 🔥 Increase filters
    x = Conv2D(num_filters, (1,1), strides=1, padding='same', 
               kernel_initializer='he_normal', kernel_regularizer=l2(5e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  # ✅ Using ReLU

    # **🔥 Extra Conv Layer Before GAP to Improve Feature Learning**
    x = Conv2D(num_filters, (3,3), padding='same', 
               kernel_initializer='he_normal', kernel_regularizer=l2(5e-5), use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)  # ✅ Using ReLU

    # **Global Average Pooling**
    x = GlobalAveragePooling2D()(x)

    # **Fully Connected Layers**
    x = Dense(256, activation='relu', kernel_regularizer=l2(5e-5))(x)  # 🔥 Increased from 128 → 256
    x = Dropout(0.4)(x)  # 🔥 Increased dropout to improve generalization

    # **Output Layer**
    outputs = Dense(1, activation='sigmoid')(x)  # ✅ Binary classification (Dog vs Not-Dog)

    # **Define Model**
    model = Model(inputs=inputs, outputs=outputs)

    return model

model = improved_tinyml_mobilenet()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), 
              loss="binary_crossentropy", metrics=["accuracy"])

# **Display Model Summary**
model.summary()
